In [1]:
!apt update -qq

35 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [2]:
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

--2023-05-26 14:16:13--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.39.144
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/downloads/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb [following]
--2023-05-26 14:16:14--  https://developer.nvidia.com/downloads/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?PwYhkpViIunk5QC4nVFBR0hlkYqGKvRmkV-1BWYcN_jukWLQRif3ITZq8

In [3]:
!dpkg -i cuda-repo-ubuntu1604–8–0-local-ga2_8.0.61–1_amd64-deb 2> /dev/null

In [4]:
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub

gpg: can't open '/var/cuda-repo-8-0-local-ga2/7fa2af80.pub': No such file or directory


In [5]:
!apt-get update -qq

In [6]:
!apt --fix-broken install
!apt-get install cuda gcc-5 g++-5 -y -qq;

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
E: Package 'gcc-5' has no installation candidate
E: Package 'g++-5' has no installation candidate


In [7]:
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package cuda-8.0
E: Couldn't find any package by glob 'cuda-8.0'


In [8]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [9]:
!apt install gcc-5 g++-5 -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package g++-5 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gcc-10-test-results gcc-9-test-results gcc-8-test-results gcc-7-test-results

Package gcc-5 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'gcc-5' has no installation candidate
E: Package 'g++-5' has no installation candidate


In [10]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [11]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [12]:
%%file version.cu
#include <thrust/version.h>
#include <iostream>

int main(void)
{
  int major = THRUST_MAJOR_VERSION;
  int minor = THRUST_MINOR_VERSION;

  std::cout << "Thrust v" << major << "." << minor << std::endl;

  return 0;
}

Writing version.cu


In [13]:
!nvcc version.cu -o version
!./version

Thrust v1.15


In [14]:
%%file thrust_example.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <algorithm>
#include <cstdlib>

int main(void)
{
  // generate 32M random numbers serially
  thrust::host_vector<int> h_vec(32 << 20);
  std::generate(h_vec.begin(), h_vec.end(), rand);

  // transfer data to the device
  thrust::device_vector<int> d_vec = h_vec;

  // sort data on the device 
  thrust::sort(d_vec.begin(), d_vec.end());

  // transfer data back to host
  thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());

  return 0;
}

Writing thrust_example.cu


In [15]:
!nvcc thrust_example.cu -o thrust_example
!./thrust_example


In [16]:
!nvprof ./thrust_example

==2810== NVPROF is profiling process 2810, command: ./thrust_example
==2810== Profiling application: ./thrust_example
==2810== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   36.35%  28.594ms         1  28.594ms  28.594ms  28.594ms  [CUDA memcpy HtoD]
                   35.97%  28.300ms         1  28.300ms  28.300ms  28.300ms  [CUDA memcpy DtoH]
                   14.39%  11.320ms         3  3.7733ms  3.7660ms  3.7769ms  void cub::DeviceRadixSortDownsweepKernel<cub::DeviceRadixSortPolicy<int, cub::NullType, int>::Policy800, bool=1, bool=0, int, cub::NullType, int>(cub::DeviceRadixSortPolicy<int, cub::NullType, int>::Policy800 const *, cub::DeviceRadixSortDownsweepKernel<cub::DeviceRadixSortPolicy<int, cub::NullType, int>::Policy800, bool=1, bool=0, int, cub::NullType, int>*, bool=1 const *, cub::DeviceRadixSortDownsweepKernel<cub::DeviceRadixSortPolicy<int, cub::NullType, int>::Policy800, bool=1, bool=0, int, cub::N

In [20]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-3ok1yhmo
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-3ok1yhmo
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=561853c2d90701fb2a749aca0fe1b2a4f1d11fa19846f30d7512abd8a082537a
  Stored in directory: /tmp/pip-ephem-wheel-cache-25181hz9/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [21]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [22]:
%%cu
#include<stdio.h>
#include<cuda.h>
__global__ void matadd(int *l,int *m, int *n)
{
    int x=blockIdx.x;
    int y=blockIdx.y;
    int id=gridDim.x * y +x;
    n[id]=l[id]+m[id];
}
int main()
{
    int a[2][3];
    int b[2][3];
    int c[2][3];
    int *d,*e,*f;
    int i,j;
    printf("\n Enter elements of first matrix of size 2 * 3\n");
    for(i=0;i<2;i++)
    {
        for(j=0;j<3;j++)
            {
                scanf("%d",&a[i][j]);
            }
    }
    printf("\n Enter elements of second matrix of size 2 * 3\n");
        for(i=0;i<2;i++)
        {
            for(j=0;j<3;j++)
                {
                    scanf("%d",&b[i][j]);
                }
        }
    cudaMalloc((void **)&d,2*3*sizeof(int));
    cudaMalloc((void **)&e,2*3*sizeof(int));
    cudaMalloc((void **)&f,2*3*sizeof(int));
 cudaMemcpy(d,a,2*3*sizeof(int),cudaMemcpyHostToDevice);
 cudaMemcpy(e,b,2*3*sizeof(int),cudaMemcpyHostToDevice);
    
dim3 grid(3,2);
/* Here we are defining two dimensional Grid(collection of blocks) structure. Syntax is dim3 grid(no. of columns,no. of rows) */

    matadd<<<grid,1>>>(d,e,f);

 cudaMemcpy(c,f,2*3*sizeof(int),cudaMemcpyDeviceToHost);
    printf("\nSum of two matrices:\n ");
    for(i=0;i<2;i++)
    {
        for(j=0;j<3;j++)
        {
              printf("%d\t",c[i][j]);
        }
        printf("\n");
    }
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);
    return 0;
}



 Enter elements of first matrix of size 2 * 3

 Enter elements of second matrix of size 2 * 3

Sum of two matrices:
 1755345044	32766	-371698876	
44084	-465744152	32756	



In [24]:
%%cu
#include<stdio.h>
#include<cuda.h>

__global__ void arradd(int *x,int *y, int *z)    //kernel definition
{
  int id=blockIdx.x; 
/* blockIdx.x gives the respective block id which starts from 0 */
  z[id]=x[id]+y[id];
}

int main()
{
    int a[6];
    int b[6];
    int c[6];
    int *d,*e,*f;
    int i;
    printf("\n Enter six elements of first array\n");
    for(i=0;i<6;i++)
    {
        scanf("%d",&a[i]);
    }
    printf("\n Enter six elements of second array\n");
        for(i=0;i<6;i++)
        {
            scanf("%d",&b[i]);
        }

/* cudaMalloc() allocates memory from Global memory on GPU */
    cudaMalloc((void **)&d,6*sizeof(int));
    cudaMalloc((void **)&e,6*sizeof(int));
    cudaMalloc((void **)&f,6*sizeof(int));

/* cudaMemcpy() copies the contents from destination to source. Here destination is GPU(d,e) and source is CPU(a,b) */
 cudaMemcpy(d,a,6*sizeof(int),cudaMemcpyHostToDevice);   
 cudaMemcpy(e,b,6*sizeof(int),cudaMemcpyHostToDevice);
 
/* call to kernel. Here 6 is number of blocks, 1 is the number of threads per block and d,e,f are the arguments */ 
arradd<<<6,1>>>(d,e,f); 

/* Here we are copying content from GPU(Device) to CPU(Host) */
 cudaMemcpy(c,f,6*sizeof(int),cudaMemcpyDeviceToHost);
    
printf("\nSum of two arrays:\n ");
    for(i=0;i<6;i++)
    {
        printf("%d\t",c[i]);
    }

/* Free the memory allocated to pointers d,e,f */
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}


 Enter six elements of first array

 Enter six elements of second array

Sum of two arrays:
 1612409572	32764	399503882	43996	-1737330968	32596	


In [26]:
%%cu 
#include "stdio.h"
int main()
{
    printf("Hello world");
 return 0;
}


Hello world


In [27]:
%%cu
#include <stdio.h>
#define N 3
__global__ void matrixMultiplication(float *A, float *B, float *C, int n)
{
int i = blockIdx.x * blockDim.x + threadIdx.x;
int j = blockIdx.y * blockDim.y + threadIdx.y;
if (i < n && j < n) {
float sum = 0.0f;
for (int k = 0; k < n; ++k) {
sum += A[i * n + k] * B[k * n + j];
}
C[i * n + j] = sum;
}
}
int main()
{
float A[N][N] = {{1, 2, 3}, {4, 5, 6}, {7, 8, 9}};
float B[N][N] = {{9, 8, 7}, {6, 5, 4}, {3, 2, 1}};
float C[N][N] = {0};
// Allocate device memory
float *d_A, *d_B, *d_C;
cudaMalloc(&d_A, N * N * sizeof(float));
cudaMalloc(&d_B, N * N * sizeof(float));
cudaMalloc(&d_C, N * N * sizeof(float));
// Copy input matrices from host to device
cudaMemcpy(d_A, A, N * N * sizeof(float), cudaMemcpyHostToDevice);
cudaMemcpy(d_B, B, N * N * sizeof(float), cudaMemcpyHostToDevice);
// Set the grid and block dimensions
dim3 gridDim(ceil(N/16.0), ceil(N/16.0), 1);
dim3 blockDim(16, 16, 1);
// Launch the kernel
matrixMultiplication<<<gridDim, blockDim>>>(d_A, d_B, d_C, N);
// Copy result matrix from device to host
cudaMemcpy(C, d_C, N * N * sizeof(float), cudaMemcpyDeviceToHost);
// Print the result matrix

printf("Result Matrix:\n");
for (int i = 0; i < N; ++i) {
for (int j = 0; j < N; ++j) {
printf("%.1f ", C[i][j]);
}
printf("\n");
}
// Free device memory
cudaFree(d_A);
cudaFree(d_B);
cudaFree(d_C);
return 0;
}

Result Matrix:
30.0 24.0 18.0 
84.0 69.0 54.0 
138.0 114.0 90.0 



In [28]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#define N 5
__global__ void add(int *a, int *b, int *c) {
int i = blockIdx.x * blockDim.x + threadIdx.x;
if (i < N) {
c[i] = a[i] + b[i];
}
}
int main() {
int a[N] = {1, 2, 3, 4, 5};
int b[N] = {6, 7, 8, 9, 10};
int c[N] = {0};
int *dev_a, *dev_b, *dev_c;
cudaMalloc((void **)&dev_a, N * sizeof(int));
cudaMalloc((void **)&dev_b, N * sizeof(int));
cudaMalloc((void **)&dev_c, N * sizeof(int));
cudaMemcpy(dev_a, a, N * sizeof(int), cudaMemcpyHostToDevice);
cudaMemcpy(dev_b, b, N * sizeof(int), cudaMemcpyHostToDevice);
add<<<1, N>>>(dev_a, dev_b, dev_c);
cudaMemcpy(c, dev_c, N * sizeof(int), cudaMemcpyDeviceToHost);
for (int i = 0; i < N; i++) {
//printf("%d ", c[i]);
printf("%d + %d = %d\n", a[i], b[i], c[i]);
}
printf("\n");
cudaFree(dev_a);
cudaFree(dev_b);
cudaFree(dev_c);
return 0;
}

1 + 6 = 7
2 + 7 = 9
3 + 8 = 11
4 + 9 = 13
5 + 10 = 15


